In [1]:
from collections import namedtuple
import re

import pdfplumber
import pandas as pd

In [2]:
jrnl_re = re.compile(r'ID (\d+) (.*) \(.*?Posted by (.*) on.*\).+(\d+ [A-Z].*)')

In [3]:
Row = namedtuple('Row', 'journal_id description username date account_name account_number debit credit')

In [21]:
def dr_cr(last_pos, row):
    return 'debit' if last_pos[row] < 830 else 'credit'

In [22]:
def numbify(num):
    return float(num.replace('$', '').replace(',', ''))

In [23]:
def page_to_df(lines, last_pos, id_info=('','','','','')):
    jrnl, journal_id, description, username, date = id_info
    rows = []

    for idx, line in enumerate(lines.split('\n')):
        if line.startswith('ID'):
            jrnl = jrnl_re.search(line)
            journal_id = jrnl.group(1)
            description = jrnl.group(2)
            username = jrnl.group(3)
            date = jrnl.group(4)

        if ') $' in line:
            *act_name, act_no, amt = line.split()
            act_name = ' '.join(act_name)
            if dr_cr(last_pos, idx) == 'debit':
                debit, credit = numbify(amt), 0
            else:
                debit, credit = 0, numbify(amt)
            rows.append(Row(journal_id, description, username, date, act_name, act_no, debit, credit))
        df = pd.DataFrame(rows)
    return df, (jrnl, journal_id, description, username, date)

In [24]:
df = pd.DataFrame()
id_info = ('','','','','')
with pdfplumber.open("Demo Company Journal Report.pdf") as pdf:
    for page in pdf.pages:
        lines = page.extract_text(x_tolerance=2, y_tolerance=0)
        words = page.extract_words(x_tolerance=2, y_tolerance=0)
        rows_dict = {b: a for a, b in enumerate(sorted(set([word['bottom'] for word in words])))}
        last_pos = {rows_dict.get(word['bottom']): word['x1'] for word in words}
        new_df, id_info = page_to_df(lines, last_pos, id_info)
        df = pd.concat([df, new_df]).reset_index(drop=True)

In [36]:
with pdfplumber.open("Demo Company Journal Report.pdf") as pdf:
    words = pdf.pages[0].extract_words(x_tolerance=2)
    rows_dict = {b: a for a, b in enumerate(sorted(set([word['bottom'] for word in words])))}
    last_pos = {rows_dict.get(word['bottom']): word['x1'] for word in words}

In [26]:
df

,journal_id,description,username,date,account_name,account_number,debit,credit
0,5985,Inventory Opening Balance (for 2019 and 2020),Michael Booner,4 May 2020,Inventory,(630),100.0,0.0
1,5985,Inventory Opening Balance (for 2019 and 2020),Michael Booner,4 May 2020,Inventory,(630),0.0,80.0
2,5985,Inventory Opening Balance (for 2019 and 2020),Michael Booner,4 May 2020,Inventory,(630),0.0,100.0
3,5985,Inventory Opening Balance (for 2019 and 2020),Michael Booner,4 May 2020,Inventory,(630),140.0,0.0
4,5985,Inventory Opening Balance (for 2019 and 2020),Michael Booner,4 May 2020,Inventory,(630),80.0,0.0
...,...,...,...,...,...,...,...,...
156,540,Melrose Parking,Jiaqi Luo,1 May 2020,Business Bank Account,(090),0.0,148.5
157,539,Payment: DIISR - Small Business Services,Jiaqi Luo,1 May 2020,Accounts Receivable,(610),0.0,2145.0
158,539,Payment: DIISR - Small Business Services,Jiaqi Luo,1 May 2020,Business Bank Account,(090),2145.0,0.0
159,538,Payment: Hamilton Smith Pty,Jiaqi Luo,1 May 2020,Business Bank Account,(090),550.0,0.0
